# Topic Modelling

This is a series of experiments to try

In [1]:
#%% Imports
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

from Predictors.sklearnclassifier import show_stats

from data.import_data import import_data, clean_data
from data.numbered_authors import NumberAuthorsTransformer
from data.padded_sentences import PaddedSentenceTransformer

Using TensorFlow backend.


In [10]:
tr, te = import_data()

In [32]:
#%% Use LDA to define topics
num_topics = 30
# Number of words in each topic
num_topic_words = 10

# Use the standard counting vectorizer.
tf_vectorizer = CountVectorizer(  # max_df=0.95, min_df=2,
#     max_features=3000,
    stop_words='english')
tf_vectors = tf_vectorizer.fit_transform(tr.text)
# Produces an array of topic->word mappings

In [33]:
lda = LatentDirichletAllocation(n_components=num_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf_vectors)

tf_feature_names = tf_vectorizer.get_feature_names()

doc_topic_distrib = lda.transform(tf_vectors)

In [34]:
def get_data_(train_limit, encoder_size, get_train_data=True):
    if get_train_data:
        # Training data
        data, _ = import_data()
    else:
        # Testing data
        _, data = import_data()

    text = list(data.text)
    author = list(data.author)

    label_enc = NumberAuthorsTransformer()
    if train_limit is None:
        train_limit = len(text)
    y = label_enc.fit_transform(author[:train_limit])

    data_enc = PaddedSentenceTransformer(encoder_size=encoder_size)
    X = data_enc.fit_transform(text[:train_limit])

    return X, y

In [35]:
#%% Init the SVM
train_limit=1000
myc = LinearSVC()
print("Getting data")
#%% Get data
X_train, y_train = get_data_(train_limit, 100)

Getting data


In [36]:
print("Training")
#%% Fit data
myc.fit(doc_topic_distrib, y_train)
print("Getting test data")
X_test, y_test = get_data_(train_limit, 100)
print("Estimating")
y_train_pred = myc.predict()
show_stats(y_test, y_train_pred)

Training


ValueError: Found input variables with inconsistent numbers of samples: [15663, 1000]